In [1]:
import twitter
import os
from config import *

# initialize api instance
twitter_api = twitter.Api(consumer_key=consumer_key,
                         consumer_secret=consumer_secret,
                         access_token_key=access_token_key,
                         access_token_secret=access_token_secret)

#print(consumer_key,consumer_secret,access_token_key,access_token_secret)

# test authentication
print(twitter_api.VerifyCredentials())
# print(twitter_api.GetStatus(126402758403305000))

{"created_at": "Wed Jul 20 02:34:27 +0000 2016", "description": "I am Al and I am passionate about technology.", "followers_count": 5, "friends_count": 52, "id": 755591669194514432, "id_str": "755591669194514432", "location": "Tustin, CA", "name": "Al", "profile_background_color": "000000", "profile_background_image_url": "http://abs.twimg.com/images/themes/theme1/bg.png", "profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_banner_url": "https://pbs.twimg.com/profile_banners/755591669194514432/1469041661", "profile_image_url": "http://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/755843006062694401/2JikSpJq_normal.jpg", "profile_link_color": "1B95E0", "profile_sidebar_border_color": "000000", "profile_sidebar_fill_color": "000000", "profile_text_color": "000000", "screen_name": "alknowstech", "status": {"created_at": "Mon Jul 31 21:38:46 +0000 2017", 

In [2]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [21]:
# test search function
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:10])

Enter a search keyword:samsung
Fetched 100 tweets for the term samsung
[{'text': 'South Korea is changing the landscape in so many ways...\n\n- Best picture: Parasite \n\n- Best boy band: BTS\n\n- Best 2… https://t.co/pkdmXY9cQx', 'label': None}, {'text': "You guys. I think Samsung is gonna shock the world and announce a new phone today. Can't be sure yet. But I'll be on site just in case.", 'label': None}, {'text': 'She looks like a dream in real life. Aowa!!! MISS UNIVERSE KE SHANDA PAH!!!! Lezothi Z!!!! Samsung just announced h… https://t.co/8mhMGsTk21', 'label': None}, {'text': "RT @darlinvante: samsung really used taehyung as their model to promote their new foldable phone he's so powerful\nhttps://t.co/C8TAS29Gck", 'label': None}, {'text': 'RT @thetaeprint: [INFO] Taehyung appeared on the screens during the live launch of Samsung’s new phone the Galaxy S20. Samsung used Taehyun…', 'label': None}, {'text': 'RT @jkmin97: Taehyung apareció durante la presentacion del nuevo telefono 

# Start preprocessing of training data

In [4]:
import pandas as pd
import numpy as np

traindata140 = os.path.join(".","trainingandtestdata_sentiment140","training.csv")
testdata140 = os.path.join(".","trainingandtestdata_sentiment140","test.csv")

In [5]:
#read the csv file command
train_df = pd.read_csv(traindata140, header=None, usecols=[0,5], names=['polarity of the tweet','text'], 
                       encoding="ISO-8859-1")
train_df.head()
# (0 = negative, 2 = neutral, 4 = positive)

,polarity of the tweet,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


# Test code

In [ ]:
test_df = pd.read_csv(testdata140, header=None, usecols=[0,5],
                      names=['polarity of the tweet', 'text'],encoding="ISO-8859-1")
conditions_test = [
    (test_df['polarity of the tweet'] == 0),
    (test_df['polarity of the tweet'] == 2),
    (test_df['polarity of the tweet'] == 4)]
choices_test = ['negative', 'neutral', 'positive']
test_df['label'] = np.select(conditions_test, choices_test)
test_df.head()

In [ ]:
test_ls = test_df.to_dict('records')

In [6]:
train_df_negative = train_df.loc[train_df['polarity of the tweet']==0]

In [7]:
train_df_positive = train_df.loc[train_df['polarity of the tweet']==4]

In [8]:
frames = [train_df_negative.iloc[0:50000,:],train_df_positive.iloc[0:50000,:]]

In [9]:
result = pd.concat(frames)

In [10]:
conditions = [
    (result['polarity of the tweet'] == 0),
    (result['polarity of the tweet'] == 2),
    (result['polarity of the tweet'] == 4)]
choices = ['negative', 'neutral', 'positive']
result['label'] = np.select(conditions, choices)
result.head()

,polarity of the tweet,text,label
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",negative
1,0,is upset that he can't update his Facebook by ...,negative
2,0,@Kenichan I dived many times for the ball. Man...,negative
3,0,my whole body feels itchy and like its on fire,negative
4,0,"@nationwideclass no, it's not behaving at all....",negative


In [11]:
train_ls = result.to_dict('records')

In [12]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import nltk
# nltk.download('punkt')

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()


In [13]:
# Process Training Set
preprocessedTrainingSet = tweetProcessor.processTweets(train_ls)

In [22]:
# Process Test Set
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

In [15]:
import nltk 

def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [16]:
def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

In [17]:
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)

In [18]:
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [19]:
import pickle
save_classifier = open("naivebayes_train_100k.pickle","wb")
pickle.dump(NBayesClassifier, save_classifier)
save_classifier.close()

In [ ]:
import pickle
classifier_f = open("naivebayes_train.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [23]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

Overall Positive Sentiment
Positive Sentiment Percentage = 90.0%


In [ ]:
NBResultLabelsPickle = [classifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

# get the majority vote
if NBResultLabelsPickle.count('positive') > NBResultLabelsPickle.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabelsPickle.count('positive')/len(NBResultLabelsPickle)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabelsPickle.count('negative')/len(NBResultLabelsPickle)) + "%")

In [ ]:
for tweet in preprocessedTrainingSet:
    print(tweet[0])

In [ ]:
trainingFeatures